In [ ]:
# import modules 
import matplotlib.cbook as cbook
from matplotlib_scalebar.scalebar import ScaleBar
import pandas as pd 
import geopandas as gpd 
import matplotlib.pyplot as plt 
import numpy as np
import PIL
import io
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
# read ESPON LOGO 
im = plt.imread('ESPON images/espon-logo.png') # insert local ESPON logo.

In [ ]:
# make the base map 
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
cities = gpd.read_file(gpd.datasets.get_path('naturalearth_cities'))

In [ ]:
fiji = world.query('name == "Fiji"')[['geometry']]

## 1. Cases

### 1.1 New cases

In [ ]:
## Read the data 
# Cases
cases = pd.read_excel('../../02_Data/02_Pandemic variables/01_Cases/EU_Cases_Weekly_Unstacked.xlsx')
cases.rename(columns = {"Unnamed: 0": 'nuts_id'}, inplace= True)
cases.index = cases.nuts_id
cases.drop('nuts_id', axis =1, inplace = True)

In [ ]:
# read shape file 
covid = gpd.read_file('../../02_Data/01_GIS/02_Full file/Full shape.shp')
# read shape file by ESPON
# shp0 = gpd.read_file('/content/drive/MyDrive/Colab Notebooks/Data/ESPON MAPS/nuts-version2016-level0.shp')
# shp1 = gpd.read_file('/content/drive/MyDrive/Colab Notebooks/Data/ESPON MAPS/nuts-version2016-level1.shp')
# shp2 = gpd.read_file('/content/drive/MyDrive/Colab Notebooks/Data/ESPON MAPS/nuts-version2016-level2.shp')
# shp3 = gpd.read_file('/content/drive/MyDrive/Colab Notebooks/Data/ESPON MAPS/nuts-version2016-level3.shp')

In [ ]:
cases.dropna(thresh = len(cases) -200, axis = 1, inplace= True)
cases = abs(cases)

In [ ]:
# turn dataframes into GeoDataFrames 
geo_cases = gpd.GeoDataFrame(pd.merge(cases, covid, left_on = cases.index, right_on = 'NUTS_ID').fillna(np.nan))


In [ ]:
geo_cases = pd.concat([geo_cases,fiji])

In [ ]:
# change the CRS to Europe 
geo_cases = geo_cases.to_crs(epsg=3035) 
world = world.to_crs(epsg = 3035)
cities = cities.to_crs(epsg = 3035)

In [ ]:
# Selecting only cities we want to see on the plot   
cities['x'] = cities['geometry'].x
cities['y'] = cities['geometry'].y
cities = cities[cities['x'] > 0.23e7]
cities = cities[cities['x'] < 0.67e7]
cities = cities[cities['y'] > 0.1e7]
cities = cities[cities['y'] < 0.545e7]
# Delete these cities from the plot 
cities = cities[~cities['name'].isin(['Beirut', 'Vatican City', 'Cairo', 'Jerusalem', 'Tripoli', 'Zagreb', 'Bratislava'])]

In [ ]:
source="""Regional level: NUTS 2/3 v2021.
Source: WHO EPISTAT, NPGEO, mscbs.gob.es.
santé-publique-France, statistichecoronavirus.it.
folkhalsomyndigheten, gov.scot, Korona.gov.sk.
data.gv.at, sso.dk, salute.gov.it."""

In [ ]:
geo_cases.describe()

In [ ]:
# Creating the bins 
bins = [-1, 0, 5, 100, 1000, 6647]
label = ['None', 'Less than 5', '5-100', '100-1000', 'More than 1000']
for col in geo_cases.iloc[:, :-2]:
    geo_cases[col] = pd.cut(geo_cases[col], bins = bins, labels= label)

In [ ]:
# set mamp general general aspect 

plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['axes.edgecolor'] = 'white'
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.4
plt.rcParams['grid.color'] = "white"
plt.rcParams['axes.facecolor'] = '#f4f9fa'

In [ ]:
## add the ESPON logo 
plt.rcParams["figure.figsize"] = [7.00, 3.50]
plt.rcParams["figure.autolayout"] = True

In [ ]:
plt.rcParams["figure.figsize"] = [7.00, 3.50]
plt.rcParams["figure.autolayout"] = True
plt.rcParams['figure.dpi'] = 80

In [ ]:
geo_cases

In [ ]:
image_frames = []

# # create the maps 
for i in geo_cases.iloc[:,:-2]:
    fig, ax = plt.subplots(figsize = (12, 12))
    # Set the global map 
    world.plot(color = '#e5e3e6', ax = ax, edgecolor = 'w', linewidth = 0.8)
    # set out map 
    geo_cases.plot(column = geo_cases[i], ax =ax, cmap = 'RdPu', legend = True, alpha = .4,
              linewidth = 0.3, edgecolor = 'w', categorical = True, missing_kwds={
            "color": "lightgrey",
            "edgecolor": "red",
            "hatch": "///",
            "label": "Missing values"})   
    
    # set the final limite
    plt.xlim(2.4e6, 6.6e6)
    plt.ylim(0.13e7, 0.545e7)
    # add capitals points 
    cities.plot(ax = ax, color = 'k', markersize = 1)
    for x, y, label in zip(cities.geometry.x, cities.geometry.y, cities.name):
        ax.annotate(label, xy=(x, y), xytext=(1, 2), textcoords="offset points", fontsize = 6.5)
    # Disable scientific representation 
    ax.ticklabel_format(useOffset=False, style='plain')

    # add the graphical-scale 
    scalebar = ScaleBar(0.66, width_fraction=None,
            location= "lower right", box_alpha = 0) # 1 pixel = 0.2 meter
    plt.gca().add_artist(scalebar)

    # legend 
    leg = ax.get_legend()
    leg.set_bbox_to_anchor((0.24, 0)) 
    leg.get_frame().set_alpha(0)
    leg.set_title('NEW CASES')
    plt.title(f'  Weekly new confirmed COVID-19 cases: {pd.Timestamp(i).month_name()} - {i.year}', fontweight = 'light',
            fontsize= 14, loc = 'left', color = '#054ea2')
    # Set outside the figure color 
    fig.patch.set_facecolor('#fbfdfe')
    #  Set source 
    fig.text(0.92, .055, source, ha='right', fontsize = 8)
    # add ESPON's title 
    plt.text(x = 3.5e6, y = 1.4e6, s = u"\u00A9 ESPON EGTC, 2021", fontsize = 10)
    # add bleu lines 
    plt.vlines(x = 2.4e6, ymin = 1e6, ymax = 2e6, color = '#054ea2', linewidth = 10)
    plt.vlines(x = 2.4e6, ymin = 4.8e6, ymax = 6e6, color = '#054ea2', linewidth = 10)
    plt.vlines(x = 6.6e6, ymin = 1e6, ymax = 6e6, color = '#054ea2', linewidth = 10)

    plt.hlines(y = 0.13e7, xmin = 5.4e6, xmax = 6.6e6, color = '#054ea2', linewidth = 5)
    plt.hlines(y = 0.545e7, xmin = 2.4e6, xmax = 3.4e6, color = '#054ea2', linewidth = 5)
    # revome axis  
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    # grid params 
    plt.tick_params(bottom = False, left = False)
    plt.grid(True, dash_capstyle = 'projecting')

    # add ESPON logo 
    newax = fig.add_axes([0.1, 0.015,0.14,0.18], anchor='NE', zorder=1)
    newax.imshow(im)
    newax.axis('off')
    plt.show()

    img = ax.get_figure()
    f = io.BytesIO()
    img.savefig(f, format= 'png')
    f.seek(0)
    image_frames.append(PIL.Image.open(f))

In [ ]:
# create a GIF animation 
image_frames[0].save('Animated Maps/Dynamic COVID-19 cases Map.gif', format = 'GIF', append_images = image_frames[1:],
                    save_all = True, duration = 800,
                     loop = 2)
f.close()

### 1.2 Cumulitave cases

## 2. Deaths

## 3. Exc. Mortality 